# CO2 forcing


From O'Connor et al. 2021:

- +1.89 W m-2

In [1]:
from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

import matplotlib.pyplot as pl
import pandas as pd
import pooch
from scipy.optimize import root

In [2]:
climate_response_df = pd.read_csv('../data/fair-calibrations/4xCO2_energy_balance_ebm3.csv')

In [3]:
scenarios = ['ssp245']
configs = ['UKESM1-0-LL']
species = ['CO2', 'CH4', 'N2O']
species, properties = read_properties(species=species)
properties['CO2']['input_mode'] = 'concentration'
properties['CH4']['input_mode'] = 'concentration'
properties['N2O']['input_mode'] = 'concentration'
properties

{'CO2': {'type': 'co2',
  'input_mode': 'concentration',
  'greenhouse_gas': True,
  'aerosol_chemistry_from_emissions': False,
  'aerosol_chemistry_from_concentration': False},
 'CH4': {'type': 'ch4',
  'input_mode': 'concentration',
  'greenhouse_gas': True,
  'aerosol_chemistry_from_emissions': False,
  'aerosol_chemistry_from_concentration': True},
 'N2O': {'type': 'n2o',
  'input_mode': 'concentration',
  'greenhouse_gas': True,
  'aerosol_chemistry_from_emissions': False,
  'aerosol_chemistry_from_concentration': True}}

In [4]:
rcmip_concentration_file = pooch.retrieve(
    url="doi:10.5281/zenodo.4589756/rcmip-concentrations-annual-means-v5-1-0.csv",
    known_hash="md5:0d82c3c3cdd4dd632b2bb9449a5c315f",
)
conc_df = pd.read_csv(rcmip_concentration_file)

In [5]:
co2 = conc_df.loc[(conc_df['Scenario']=='ssp245')&(conc_df['Variable']=='Atmospheric Concentrations|CO2')&(conc_df['Region']=='World'),'1850':'2015'].values.squeeze()
co2[-2] = co2[-1]
ch4 = conc_df.loc[(conc_df['Scenario']=='historical')&(conc_df['Variable']=='Atmospheric Concentrations|CH4')&(conc_df['Region']=='World'),'1850'].values[0]
n2o = conc_df.loc[(conc_df['Scenario']=='historical')&(conc_df['Variable']=='Atmospheric Concentrations|N2O')&(conc_df['Region']=='World'),'1850'].values[0]

In [6]:
target_forcing = 1.89

def erf_rootfinder(x):
    f = FAIR(temperature_prescribed=True)
    f.define_time(1850, 2015, 1)
    f.define_scenarios(scenarios)
    f.define_configs(configs)
    f.define_species(species, properties)
    f.allocate()
    f.fill_species_configs()
    
    # 1850 baselines
    f.species_configs['baseline_concentration'].loc[dict(specie='CO2')] = co2[0]
    f.species_configs['baseline_concentration'].loc[dict(specie='CH4')] = ch4
    f.species_configs['baseline_concentration'].loc[dict(specie='N2O')] = n2o
    
#    f.fill_species_configs()
    fill(f.climate_configs['ocean_heat_capacity'], climate_response_df.loc[0, 'C1':'C3'])
    fill(f.climate_configs['ocean_heat_transfer'], climate_response_df.loc[0, 'kappa1':'kappa3'])
    fill(f.climate_configs['deep_ocean_efficacy'], climate_response_df.loc[0, 'epsilon'])
    fill(f.climate_configs['gamma_autocorrelation'], climate_response_df.loc[0, 'gamma'])
    
    # fill emissions and concentrations
    f.concentration.loc[dict(specie='CO2')] = co2[:, None, None]
    f.concentration.loc[dict(specie='CH4')] = ch4
    f.concentration.loc[dict(specie='N2O')] = n2o

    f.temperature[:] = 0

    # this is the variable we are changing: CO2 scale factor
    f.species_configs['forcing_scale'].loc[dict(specie="CO2")] = x
    
    initialise(f.forcing, 0)
    initialise(f.temperature, 0)
    initialise(f.cumulative_emissions, 0) 
    f.run(progress=False)
    return f.forcing[-1, 0, 0, 0] - target_forcing

In [7]:
sol = root(erf_rootfinder, 1)

In [8]:
sol.x[0]

0.9935419214400296

In [9]:
co2

array([284.3169988 , 284.45100403, 284.59800212, 284.73099772,
       284.84599813, 284.94099935, 285.04899343, 285.20399984,
       285.36900075, 285.54499563, 285.73899333, 285.9330012 ,
       286.10000102, 286.27100118, 286.44200134, 286.61399587,
       286.78099823, 286.95500183, 287.10500081, 287.22499847,
       287.35499827, 287.49400075, 287.66399638, 287.86000061,
       288.06099955, 288.29100291, 288.51999664, 288.75200144,
       288.9930013 , 289.22100067, 289.46999868, 289.73699697,
       290.01900228, 290.26300049, 290.51199849, 290.79699707,
       291.09999593, 291.41399892, 291.76300049, 292.11300151,
       292.45799764, 292.81599935, 293.16699982, 293.47699992,
       293.79099782, 294.07899729, 294.36499786, 294.64599864,
       294.95399984, 295.29999797, 295.67499542, 296.0069987 ,
       296.32499949, 296.65399679, 296.95399475, 297.28899638,
       297.66199748, 298.09799449, 298.51799011, 298.93599192,
       299.37699636, 299.82899221, 300.35299174, 300.90